# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f139ffb0af0>
├── 'a' --> tensor([[-0.5121, -0.8929,  0.2321],
│                   [ 0.7583, -0.1613, -0.8999]])
└── 'x' --> <FastTreeValue 0x7f139ff9c2b0>
    └── 'c' --> tensor([[ 1.7852, -2.2130,  0.1380, -0.8892],
                        [-2.3286, -0.8039, -0.1682, -0.2360],
                        [ 0.5821, -0.5452, -0.2539, -0.6129]])

In [4]:
t.a

tensor([[-0.5121, -0.8929,  0.2321],
        [ 0.7583, -0.1613, -0.8999]])

In [5]:
%timeit t.a

63 ns ± 0.0776 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f139ffb0af0>
├── 'a' --> tensor([[ 0.4015, -0.7458, -0.2354],
│                   [ 1.3399, -0.3378,  0.1665]])
└── 'x' --> <FastTreeValue 0x7f139ff9c2b0>
    └── 'c' --> tensor([[ 1.7852, -2.2130,  0.1380, -0.8892],
                        [-2.3286, -0.8039, -0.1682, -0.2360],
                        [ 0.5821, -0.5452, -0.2539, -0.6129]])

In [7]:
%timeit t.a = new_value

65.4 ns ± 0.0767 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.5121, -0.8929,  0.2321],
               [ 0.7583, -0.1613, -0.8999]]),
    x: Batch(
           c: tensor([[ 1.7852, -2.2130,  0.1380, -0.8892],
                      [-2.3286, -0.8039, -0.1682, -0.2360],
                      [ 0.5821, -0.5452, -0.2539, -0.6129]]),
       ),
)

In [10]:
b.a

tensor([[-0.5121, -0.8929,  0.2321],
        [ 0.7583, -0.1613, -0.8999]])

In [11]:
%timeit b.a

61.8 ns ± 0.0894 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 1.5200,  1.5257, -1.2147],
               [-1.7584,  0.3965, -1.0849]]),
    x: Batch(
           c: tensor([[ 1.7852, -2.2130,  0.1380, -0.8892],
                      [-2.3286, -0.8039, -0.1682, -0.2360],
                      [ 0.5821, -0.5452, -0.2539, -0.6129]]),
       ),
)

In [13]:
%timeit b.a = new_value

487 ns ± 0.0657 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

865 ns ± 21.4 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.3 µs ± 17 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

137 µs ± 1.48 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

138 µs ± 501 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f13c88550a0>
├── 'a' --> tensor([[[-0.5121, -0.8929,  0.2321],
│                    [ 0.7583, -0.1613, -0.8999]],
│           
│                   [[-0.5121, -0.8929,  0.2321],
│                    [ 0.7583, -0.1613, -0.8999]],
│           
│                   [[-0.5121, -0.8929,  0.2321],
│                    [ 0.7583, -0.1613, -0.8999]],
│           
│                   [[-0.5121, -0.8929,  0.2321],
│                    [ 0.7583, -0.1613, -0.8999]],
│           
│                   [[-0.5121, -0.8929,  0.2321],
│                    [ 0.7583, -0.1613, -0.8999]],
│           
│                   [[-0.5121, -0.8929,  0.2321],
│                    [ 0.7583, -0.1613, -0.8999]],
│           
│                   [[-0.5121, -0.8929,  0.2321],
│                    [ 0.7583, -0.1613, -0.8999]],
│           
│                   [[-0.5121, -0.8929,  0.2321],
│                    [ 0.7583, -0.1613, -0.8999]]])
└── 'x' --> <FastTreeValue 0x7f13c88bac70>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

33.2 µs ± 95.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f13c8859880>
├── 'a' --> tensor([[-0.5121, -0.8929,  0.2321],
│                   [ 0.7583, -0.1613, -0.8999],
│                   [-0.5121, -0.8929,  0.2321],
│                   [ 0.7583, -0.1613, -0.8999],
│                   [-0.5121, -0.8929,  0.2321],
│                   [ 0.7583, -0.1613, -0.8999],
│                   [-0.5121, -0.8929,  0.2321],
│                   [ 0.7583, -0.1613, -0.8999],
│                   [-0.5121, -0.8929,  0.2321],
│                   [ 0.7583, -0.1613, -0.8999],
│                   [-0.5121, -0.8929,  0.2321],
│                   [ 0.7583, -0.1613, -0.8999],
│                   [-0.5121, -0.8929,  0.2321],
│                   [ 0.7583, -0.1613, -0.8999],
│                   [-0.5121, -0.8929,  0.2321],
│                   [ 0.7583, -0.1613, -0.8999]])
└── 'x' --> <FastTreeValue 0x7f13c8855850>
    └── 'c' --> tensor([[ 1.7852, -2.2130,  0.1380, -0.8892],
                        [-2.3286, -0.8039, -0.1682, -0.2360],
                 

In [23]:
%timeit t_cat(trees)

30.5 µs ± 80.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

59.4 µs ± 44.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.5121, -0.8929,  0.2321],
                [ 0.7583, -0.1613, -0.8999]],
       
               [[-0.5121, -0.8929,  0.2321],
                [ 0.7583, -0.1613, -0.8999]],
       
               [[-0.5121, -0.8929,  0.2321],
                [ 0.7583, -0.1613, -0.8999]],
       
               [[-0.5121, -0.8929,  0.2321],
                [ 0.7583, -0.1613, -0.8999]],
       
               [[-0.5121, -0.8929,  0.2321],
                [ 0.7583, -0.1613, -0.8999]],
       
               [[-0.5121, -0.8929,  0.2321],
                [ 0.7583, -0.1613, -0.8999]],
       
               [[-0.5121, -0.8929,  0.2321],
                [ 0.7583, -0.1613, -0.8999]],
       
               [[-0.5121, -0.8929,  0.2321],
                [ 0.7583, -0.1613, -0.8999]]]),
    x: Batch(
           c: tensor([[[ 1.7852, -2.2130,  0.1380, -0.8892],
                       [-2.3286, -0.8039, -0.1682, -0.2360],
                       [ 0.5821, -0.5452, -0.2539, -0.6129]],
         

In [26]:
%timeit Batch.stack(batches)

78.7 µs ± 202 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.5121, -0.8929,  0.2321],
               [ 0.7583, -0.1613, -0.8999],
               [-0.5121, -0.8929,  0.2321],
               [ 0.7583, -0.1613, -0.8999],
               [-0.5121, -0.8929,  0.2321],
               [ 0.7583, -0.1613, -0.8999],
               [-0.5121, -0.8929,  0.2321],
               [ 0.7583, -0.1613, -0.8999],
               [-0.5121, -0.8929,  0.2321],
               [ 0.7583, -0.1613, -0.8999],
               [-0.5121, -0.8929,  0.2321],
               [ 0.7583, -0.1613, -0.8999],
               [-0.5121, -0.8929,  0.2321],
               [ 0.7583, -0.1613, -0.8999],
               [-0.5121, -0.8929,  0.2321],
               [ 0.7583, -0.1613, -0.8999]]),
    x: Batch(
           c: tensor([[ 1.7852, -2.2130,  0.1380, -0.8892],
                      [-2.3286, -0.8039, -0.1682, -0.2360],
                      [ 0.5821, -0.5452, -0.2539, -0.6129],
                      [ 1.7852, -2.2130,  0.1380, -0.8892],
                      [-2.3286, -

In [28]:
%timeit Batch.cat(batches)

145 µs ± 210 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

306 µs ± 1.7 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
